# Ergast API Data Ingestion

In [2]:
import sys
import os
import pandas as pd
from datetime import datetime


module_path = os.path.abspath(os.path.join("..", "scripts"))
if module_path not in sys.path:
    sys.path.append(module_path)

from f1_data_fetcher import (
    fetch_all_race_data,
    fetch_all_race_results,
    fetch_all_f1_drivers,
    fetch_all_f1_circuits,
    fetch_all_driver_standings,
)

In [3]:
START_YEAR = 1995
END_YEAR = datetime.now().year

## API Requests

In [ ]:
# Fetch the data and create a DataFrame
race_data_df = fetch_all_race_data(START_YEAR, END_YEAR)
race_data_df.to_csv(
    f"../data/raw/Race_Information_{START_YEAR}_{END_YEAR}.csv", index=False
)

In [ ]:
race_results_data_df_list = fetch_all_race_results(race_data_df, START_YEAR, END_YEAR)
race_results_data_df = pd.concat(race_results_data_df_list, ignore_index=True)
race_results_data_df.to_csv(
    f"../data/raw/Race_Results_{start_year}_{end_year}.csv", index=False
)

In [ ]:
# Fetch all F1 drivers
all_drivers_df = fetch_all_f1_drivers()
all_drivers_df.to_csv("../data/raw/drivers_information.csv",index=False)

In [ ]:
# Fetch all F1 circuits
all_circuits_df = fetch_all_f1_circuits()
all_circuits_df.to_csv("../data/raw/circuits_information.csv", index=False)

In [ ]:
# Fetch driver and constructor standings
driver_standing_df = fetch_all_driver_standings(race_data_df)
driver_standing_df.to_csv("../data/raw/driver_standings.csv", index=False)